# Trabalho prático II - Classificação

O objetivo deste trabalho é praticar os conceitos de aprendizado supervisionado que vimos em sala.

A sua tarefa será treinar um classificador para um conjunto de dados misterioso (se eu falasses qual é o objetivo do modelo, você encontraria soluções na Internet).

Baixe o arquivo [t2.tar.gz](https://drive.google.com/file/d/125plHKUzFGxHjjCiVJcTQG2bPG_zgDNV/view?usp=sharing). Descoprima este arquivo para encontrar outros quatro. Os arquivos `train_X.csv` e `train_y.csv` possuem os dados que você deve usar para treinar o modelo. O arquivo `test_X.csv` possui os objetos para os quais você deve encontrar as classes (testar o modelo). Por fim, o arquivo `test_example_y.csv` é um exemplo de como sua solução final deve ser organizada.

Cada linha dos arquivos `train_X.csv` e `test_X.csv` tem 15 campos descrevendo um objeto misterioso. O campo `id` representa o identificador do objeto, sendo que este campo não deve ser considerado em seu modelo. Os atributos a serem usados no modelo são os 14 campos rotulados de de `a` até `n`. Desses atributos:
- `b`, `d`, `f`, `g`, `h`, `i`, `j` e `n` são categóricos; e
- `a`, `c`, `e`, `k`, `l` e `m` são numéricos.

Cada linha do arquivo `train_y.csv` possui dois campos. O primeiro é o identificador de um objeto do arquivo `train_X.csv` e o segundo é a classe do respectivo objeto (0 ou 1).

Seu objetivo é encontrar as classes dos objetos do arquivo `test_X.csv` e mostrar como chegou em sua solução! Os dados do arquivo de teste foram obtido a partir de uma amostra aleatório do todo. Ou seja, um modelo bem treinado, e sem _overfitting_, em `train_X.csv` e `train_y.csv` se sairá bem em `test_X.csv`.

**Data de entrega:** dia 4 de julho de 2018.

**Grupo:** de até 3 pessoas, mas duas pessoas do mesmo grupo no trabalho 1 não podem pertencer ao mesmo grupo nesse trabalho.

**Valor:** 20% da nota do semestre.

Os três seguintes pontos descrevem o que obrigatoriamente deve ser entregue, com seu respectivo valor.

1 - **[10 pontos]** Este notebook com todo seu código e resultados (números, tabelas e gráficos). Você pode usar qualquer um dos métodos que estudamos ou alguma de suas variações próximas. Se estiver na dúvida se pode usar um método, basta perguntas no Piazza. Comentários e justificativas no notebook não serão considerados para sua nota.
O notebook deve ser enviado para o email do professor.

2 - **[8 pontos]** Um relatório digitado contendo: capa, introdução, metodologia, resultados, conclusão e referências. O relatório deve ter no máximo 10 páginas, com coluna simples, fonte 11, espaçamento 1.5 e margens de 2cm. A seção de metodologia deve conter uma descrição detalhada dos passos seguidos (não incluir código no relatório). A seção de resultados deve conter obrigatoriamente: uma caracterização descritiva dos dados, matriz de confusão das predições, _precision_, _recall_, _F1 score_ e acurácia. Todas as métricas de predição devem ser calculadas a partir dos arquivos de treinamento por meio de validação cruzada.
O relatório deve ser enviado para o email do professor.

3 - **[2 pontos + equivalente a lista extra pela classificação]** A sua predição final do arquivo `test_X.csv` deve ser enviada para o professor por email. O formato deve ser o mesmo do arquivo `train_y.csv`, assim como exemplificado em `test_example_y.csv` (mas repare que as classes desse último arquivo foram gerados de forma aleatória). Em outras palavras, o arquivo a ser entregue deve ter dois campos. O campo `id` é o identificador do objeto em `test_X.csv` e o campo `label` é a classe que seu modelo encontrou para o objeto em questão. A primeira linha do arquivo deve conter os nomes das colunas.
A entrega desse arquivo é obrigatória e vale dois pontos. Além disso, o trabalho com maior _F1 score_ ganhará o equivalente a 100% de uma lista extra. O trabalho com o pior _F1 score_ não ganhará nota extra alguma. Os demais trabalhos terão nota proporcional.
O professor se reserva o direito de anular esse quesito (nota extra) se houver indícios de má conduta durante a competição.

**Kaggle:** Estou tentando criar uma competição para esse trabalho na plataforma _Kaggle_. Se eu conseguir, compartilho o _link_ com você no _Piazza_.

# Bibliotecas importadas

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np
import matplotlib.pyplot as plt
import time

# Leitura dos arquivos de treinamento e teste

In [2]:
df_train_values = pd.read_csv("data/train_X.csv")
df_train_labels = pd.read_csv("data/train_Y.csv")
ids = df_train_values["id"]

df_test_values = pd.read_csv("data/test_X.csv")

df_train_values = df_train_values.drop(columns=["id"])
df_train_labels = df_train_labels.drop(columns=["id"])
df_test_values = df_test_values.drop(columns=["id"])

# Tratando arquivos

In [3]:
def categorical_attributes(df,cols):
    for char in cols:
        df = pd.concat(
            [df,pd.get_dummies(df[char], prefix=char)],axis=1)
        df.drop([char],axis=1, inplace=True)
    return df

cols = ["b", "d", "f", "g", "h", "i", "j", "n"]

df_train_values = categorical_attributes(df_train_values,cols)
df_test_values = categorical_attributes(df_test_values,cols)


In [71]:
x_train, x_test, y_train, y_test = train_test_split(df_train_values, df_train_labels, test_size=0.33)

In [72]:
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [73]:
i = 3
n_estimators = [3000]
clf = RandomForestClassifier()

In [79]:
start = time.time()

gscv = GridSearchCV(clf, param_grid = {'n_estimators': n_estimators}, cv = i, n_jobs=-1)
gscv.fit(x_train,y_train)
y_pred = gscv.predict(x_test)
end = time.time()

print('Default Scoring done in ' + str(end - start) + ' seconds!')
print("Accuracy: %0.2f" % (accuracy_score(y_pred, y_test)))

Default Scoring done in 435.9885401725769 seconds!
Accuracy: 0.85


In [80]:
start = time.time()

gscv = GridSearchCV(clf, scoring = 'precision', param_grid = {'n_estimators': n_estimators}, cv = i, n_jobs=-1)
gscv.fit(x_train,y_train)
y_pred = gscv.predict(x_test)
end = time.time()

print('Precision Scoring done in ' + str(end - start) + ' seconds!')
print("Precision: %0.2f" % (precision_score(y_pred, y_test)))

Precision Scoring done in 413.04309010505676 seconds!
Precision: 0.63


In [81]:
start = time.time()

gscv = GridSearchCV(clf, scoring = 'recall', param_grid = {'n_estimators': n_estimators}, cv = i, n_jobs=-1)
gscv.fit(x_train,y_train)
y_pred = gscv.predict(x_test)
end = time.time()

print('Recall Scoring done in ' + str(end - start) + ' seconds!')
print("Recall: %0.2f" % (recall_score(y_pred, y_test)))

Recall Scoring done in 415.09074091911316 seconds!
Recall: 0.73


In [82]:
start = time.time()

gscv = GridSearchCV(clf, scoring = 'f1_macro', param_grid = {'n_estimators': n_estimators}, cv = i, n_jobs=-1)
gscv.fit(x_train,y_train)
y_pred = gscv.predict(x_test)
end = time.time()

print('F1 Scoring done in ' + str(end - start) + ' seconds!')
print("F1 Scoring: %0.2f" % ((f1_score(y_pred, y_test))))

F1 Scoring done in 402.0065186023712 seconds!
F1 Scoring: 0.68


In [ ]:
# start = time.time()
# cValues = np.logspace(-10, 10, 100)
# # clf = GridSearchCV(svm.SVC(kernel = 'linear'), param_grid = {'C': cValues}, n_jobs=-1)
# # clf.fit(x_train, y_train)
# clf = RandomForestClassifier(n_estimators=10000, n_jobs = -1)
# clf.fit(x_train, y_train)
# endTraining = time.time()
# print('Training done in ' + str(endTraining - start) + ' seconds!')

In [ ]:
# start = time.time()
# cValues = np.logspace(-10, 10, 100)
# clf = GridSearchCV(svm.SVC(kernel = 'linear'), param_grid = {'C': cValues}, n_jobs=-1)
# clf.fit(x_train, y_train)
# endTraining = time.time()
# print('Training done in ' + str(endTraining - start) + ' seconds!')

In [ ]:
# df_answer_labels = clf.predict(df_train_values)
# endEvaluation = time.time()
# print(accuracy_score(df_train_labels, df_answer_labels))
# print('Evaluation done in ' + str(endEvaluation - endTraining) + ' seconds!')
# # cnf_matrix = confusion_matrix(y_test, y_pred)